In [3]:

import pandas as pd
import csv
import torch
from torch import nn
import numpy as np
import re
# import spacy
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torchvision.transforms as T
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
import spacy
from spacy.lang.en import English
from emoji import UNICODE_EMOJI
from util import *
from torch.utils.data import DataLoader
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")

print(torch.cuda.is_available())
file_path = 'Train_Tagged_Titles.tsv'
df = pd.read_csv(file_path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)

from dateutil.parser import parse


wv_from_bin = load_embedding_model()
title_path = 'Listing_Titles.tsv'
df_title = pd.read_csv(title_path, sep="\t", dtype=str, keep_default_na=False, na_values=[""], quoting=csv.QUOTE_NONE)
df_title.head()

titles_x = []
vocab = set()
MAX_LEN = 0 
for index, row in df_title.iterrows():
    print(index,end='\r')
    sent_list = row['Title'].split(' ')
    MAX_LEN = max(MAX_LEN, len(sent_list))
    vocab.update(sent_list)
    if index== 30000:
        break
df = df.fillna('NAN')

x = []
y = []
for i in range(1,5001):
    temp_df = df[df['Record Number']==str(i)]
    tokens = []
    tages = []
    
    for index, row in temp_df.iterrows():
        if row['Tag'] == 'NAN':
            tokens.append(row['Token'])
            tages.append(temp)
        else:
            temp =  row['Tag']
            tokens.append(row['Token'])
            tages.append(row['Tag'])

    vocab.update(tokens)
    
    # doc = nlp(sentence)
    # sentence = doc._.outcome_spellCheck
    # tokens = sentence.split(' ')
    x.append(tokens)
    y.append(tages)
    
MAX_LEN = 0 
for i in x:
    MAX_LEN = max(len(i), MAX_LEN)
MAX_LEN +=20

target = set()
for i in y:
    target.update(i)
    
target_name_to_index = {}

num = 1

for i in target:
    target_name_to_index[i]= num
    target_name_to_index[num]= i
    num+=1
    
target_name_to_index[0] = '[PAD]'
target_name_to_index['[PAD]'] = 0

CAT_NUM = int(len(target_name_to_index)/2)

vector_space = []
NUM_COMPONENTS= 290 #you can select any number of components.
### Vocabulary cleaning



for i in vocab:
    w = vocab_clean(i)
    vector_space.append(wv_from_bin[w])




          

True


In [37]:
#### Padding is a zero vector. 
wv_from_bin['[PAD]'] = np.zeros(300)+0.1

In [285]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.00001, random_state=48)


In [286]:
for i in range(len(X_train)):
    Pad_token  = 'NULL'
    X_train[i]= [Pad_token,Pad_token] + X_train[i] + [Pad_token,Pad_token] 
for i in range(len(X_test)):
    X_test[i]= [Pad_token,Pad_token] + X_test[i] + [Pad_token,Pad_token] 

#### Each input is a concatnation of three vectors. The vector of the word we are predicting the tag of, vector of the word before that word and vector of the word after that word. The vectors are weighted differently. 

In [287]:
NUM_COMPONENTS= 300
x_train_vectorized = []


for sentence_ind in range(len(X_train)):
    for word_ind in range(2,len(X_train[sentence_ind])-2):
            vector1 = wv_from_bin[vocab_clean(X_train[sentence_ind][word_ind-2])]*0.7
            vector2 = wv_from_bin[vocab_clean(X_train[sentence_ind][word_ind-1])]*0.9
            vector3 = wv_from_bin[vocab_clean(X_train[sentence_ind][word_ind])]
            vector4 = wv_from_bin[vocab_clean(X_train[sentence_ind][word_ind+1])]*0.9
            vector5 = wv_from_bin[vocab_clean(X_train[sentence_ind][word_ind+2])]*0.7
            x_train_vectorized.append(np.array([vector1,vector2,vector3,vector4,vector5]).flatten())        
        

x_test_vectorized = []
for sentence_ind in range(len(X_test)):
    for word_ind in range(2,len(X_test[sentence_ind])-2):
        vector1 = wv_from_bin[vocab_clean(X_test[sentence_ind][word_ind-2])]*0.7
        vector2 = wv_from_bin[vocab_clean(X_test[sentence_ind][word_ind-1])]*0.9
        vector3 = wv_from_bin[vocab_clean(X_test[sentence_ind][word_ind])]
        vector4 = wv_from_bin[vocab_clean(X_test[sentence_ind][word_ind+1])]*0.6
        vector5 = wv_from_bin[vocab_clean(X_test[sentence_ind][word_ind+2])]*0.5
        x_test_vectorized.append(np.array([vector1,vector2,vector3,vector4,vector5]).flatten())        
            
x_test_vectorized = np.array(x_test_vectorized)
x_train_vectorized = np.array(x_train_vectorized)

print(x_train_vectorized.shape)
print(x_test_vectorized.shape)



y_digit_train = []
for sentence_ind in range(len(y_train)):
    for tag_ind in range(len(y_train[sentence_ind])):
        y_digit_train.append(target_name_to_index[y_train[sentence_ind][tag_ind]])

y_digit_train = np.array(y_digit_train)
print(y_digit_train.shape)

y_digit_test  = []
for sentence_ind in range(len(y_test)):
    for tag_ind in range(len(y_test[sentence_ind])):
        y_digit_test.append(target_name_to_index[y_test[sentence_ind][tag_ind]])
y_digit_test = np.array(y_digit_test)
print(y_digit_test.shape)

(55108, 1500)
(13, 1500)
(55108,)
(13,)


In [288]:
train_dataset = []
for i in range(len(x_train_vectorized)):
    train_dataset.append((x_train_vectorized[i],
                          y_digit_train[i]))
    
test_dataset = []
for i in range(len(x_test_vectorized)):
    test_dataset.append((x_test_vectorized[i],
                          y_digit_test[i]))
    
print(len(test_dataset))

13


In [2]:
NUM_COMPONENT=300
class LSTMTagger(nn.Module):


    def __init__(self, batch_size=64):
        '''
        hidden_dim: can be anything, usually 32 or 64.
        vocab_size: vocabulary size includes an index for padding.
        output_size: We need to exclude the index for padding here.
        '''
        super().__init__()
        self.l_relu =  nn.LeakyReLU(0.02)
        self.fc1= nn.Linear(NUM_COMPONENTS*5, 2048)
        self.bn05 = nn.BatchNorm1d(2048)
        self.fc2 = nn.Linear(2048, 2048)
        self.bn1 = nn.BatchNorm1d(2048)
        self.fc3 = nn.Linear(2048, 2048)
        self.bn2 = nn.BatchNorm1d(2048)
        self.fc4 = nn.Linear(2048, 2048)
        self.bn3 = nn.BatchNorm1d(2048)
        self.fc5 = nn.Linear(2048, 2048)
        self.bn4 = nn.BatchNorm1d(2048)
        self.fc5 = nn.Linear(2048, 2048)
        self.bn4 = nn.BatchNorm1d(2048)
        
        # self.fc6 = nn.Linear(2048, 2048)
        # self.bn5 = nn.BatchNorm1d(2048)
            
        # self.fc7 = nn.Linear(2048, 2048)
        # self.bn6 = nn.BatchNorm1d(2048) 
           
        # self.fc8 = nn.Linear(2048, 1024)
        # self.bn7 = nn.BatchNorm1d(1024)        
        
        self.fc9 = nn.Linear(2048, 33)
        self.bn8 = nn.BatchNorm1d(33)
        
        
        
        
        self.fc10 = nn.Linear(33, CAT_NUM)
        self.fc14 = nn.Linear(2048, 512)
        
        self.drop = nn.Dropout(0.05)
        
    def forward(self, words):
        '''
        Parameters
        ----------
        sentences: padded sentences tensor. Each element of the tensor is an array of words.
        X_lengths: length of sentence tensor. Each element of the tensor is the original
                   length of the unpadded sentence.
        Returns
        -------
        '''
        # Dimensions of tensors:
        # (Note that seq_len is max length)
        # Shape of embedding (embeds): batch_size, seq_len, hidden_dim
        # Shape of embedding post packing (embeds): batch_size, orig_len, hidden_dim
        # Shape of self.hidden: (num_layers*num_directions, batch_size, hidden_dim)
        # Shape of lstm_out: batch_size, seq_len, hidden_dim
        # Shape of tag_scores: batch_size, 1
        



        out14 = self.fc1(words)
        out14 = self.bn05(out14)
        out = self.l_relu(out14)
        
        out = self.fc2(out)
        out = self.bn1(out)
        out = self.l_relu(out)
        # out = self.drop(out)
            
        out = self.fc3(out)
        out = self.bn2(out)
        out = self.l_relu(out)
        # out = self.drop(out)
                
        out = self.fc4(out)
        out = self.bn3(out)
        out = self.l_relu(out)
        # out = self.drop(out)
        
        out = self.fc5(out)
        out = self.bn4(out)
        out = self.l_relu(out)
        # out = self.drop(out)        
        # out = self.fc6(out)
        # out = self.bn5(out)
        # out = self.l_relu(out)

        # out = self.fc7(out)
        # out = self.bn6(out)
        # out = self.l_relu(out)
        
        # out = self.fc8(out)
        # out = self.bn7(out)
        # out = self.l_relu(out)
        
        out = self.fc9(out)
        out = self.bn8(out)
        out = self.l_relu(out)
        
        out = self.fc10(out)
        return out

NameError: name 'nn' is not defined

In [290]:
BATCHSIZE = 1024
print("MAX LENTH:",MAX_LEN)
print("NUMBER OF PRINCEPLE COMPONENT:",NUM_COMPONENTS)
print("NUMBER OF CATAGORIES:",CAT_NUM)
device = torch.device("cuda")
network = LSTMTagger(batch_size = BATCHSIZE)
model = network.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.5E-10)
scheduler = StepLR(optimizer, step_size=10, gamma=0.93)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=BATCHSIZE)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=BATCHSIZE)
loss_fn =  nn.CrossEntropyLoss( reduction='sum')
def train(model, device, train_loader, optimizer, epoch, loss_fn,batch_size, log_interval=400):
    model.train()
    lose = 0 
    for batch_idx, (data, target) in enumerate(train_loader):
        batch_size = data.size()[0]
        data, target = data.to(device).type(torch.cuda.FloatTensor), target.to(device).type(torch.cuda.LongTensor)
        
        optimizer.zero_grad()
        
        y_pred = model.forward(data)
        
        loss =  F.nll_loss(y_pred,
                        target)
        loss.backward()
        optimizer.step()
        lose += loss.item()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    return lose/len(train_loader.dataset)

def test(model, device, test_loader, batch_size):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            batch_size = data.size()[0]
            data, target = data.to(device).type(torch.cuda.FloatTensor), target.type(torch.cuda.LongTensor)
            y_pred = model.forward(data)
            test_loss +=  F.nll_loss(y_pred,
                            target)
            pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return correct / len(test_loader.dataset)


MAX LENTH: 42
NUMBER OF PRINCEPLE COMPONENT: 300
NUMBER OF CATAGORIES: 33


In [291]:
train_e=[]
test_e = []
for i in range(100):
      
    train_e.append(train(model, device, train_loader,
          optimizer, i,loss_fn=loss_fn,
          batch_size=BATCHSIZE))
    accuracy_ = test(model, device, test_loader, batch_size=BATCHSIZE)
    print(accuracy_)
#     if accuracy_ > 0.99:
#           break 
    test_e.append(accuracy_)
    scheduler.step()

Train Epoch: 1 [0/55108 (0%)]	Loss: 0.047022

Test set: Average loss: -0.1065, Accuracy: 9/13 (69%)

0.6923076923076923
Train Epoch: 2 [0/55108 (0%)]	Loss: -1.531150

Test set: Average loss: -0.1223, Accuracy: 10/13 (77%)

0.7692307692307693
Train Epoch: 3 [0/55108 (0%)]	Loss: -1.720946

Test set: Average loss: -0.1241, Accuracy: 10/13 (77%)

0.7692307692307693
Train Epoch: 4 [0/55108 (0%)]	Loss: -1.880757

Test set: Average loss: -0.1282, Accuracy: 10/13 (77%)

0.7692307692307693
Train Epoch: 5 [0/55108 (0%)]	Loss: -2.029626

Test set: Average loss: -0.1335, Accuracy: 10/13 (77%)

0.7692307692307693
Train Epoch: 6 [0/55108 (0%)]	Loss: -2.168331

Test set: Average loss: -0.1405, Accuracy: 10/13 (77%)

0.7692307692307693
Train Epoch: 7 [0/55108 (0%)]	Loss: -2.297377

Test set: Average loss: -0.1505, Accuracy: 11/13 (85%)

0.8461538461538461
Train Epoch: 8 [0/55108 (0%)]	Loss: -2.419702

Test set: Average loss: -0.1589, Accuracy: 11/13 (85%)

0.8461538461538461
Train Epoch: 9 [0/55108 (0

In [292]:
def predict_tag(sample):
    sample = [vocab_clean(word) for word in sample]
    sentence = []
    sample = [Pad_token,Pad_token] + sample + [Pad_token,Pad_token] 
    for word_ind in range(2,len(sample)-2):
        vector1 = wv_from_bin[vocab_clean(sample[word_ind-2])]*0.7
        vector2 = wv_from_bin[vocab_clean(sample[word_ind-1])]*0.9
        vector3 = wv_from_bin[vocab_clean(sample[word_ind])]
        vector4 = wv_from_bin[vocab_clean(sample[word_ind+1])]*0.6
        vector5 = wv_from_bin[vocab_clean(sample[word_ind+2])]*0.5
        sentence.append(np.array([vector1,vector2,vector3,vector4,vector5]).flatten()) 
    sentence = np.array(sentence)

    out_list = []
    for word_ind in range(sentence.shape[0]):
        word = sentence[word_ind,:]
        sample = torch.tensor(word).to(device).type(torch.cuda.FloatTensor)
        model.eval()
        optimizer.zero_grad()
                
        y_pred = model.forward(sample.view(1,5*300))
        y_pred = y_pred.cpu()
        y_pred = y_pred.detach().numpy()
        y_pred = y_pred.squeeze()

        y_pred = y_pred.argmax(axis = 0)

        out_list.append(target_name_to_index[y_pred])

    return out_list

def tokenizer(title):
    tokens = title.split(' ')
    out = []
    for token in tokens:
        out.append(token)
    return out

submission = {'Record number':[], "Aspect Name":[], "Aspect Value":[]}
for num in range(5001,30001):
    print(int(((num-5000)/(30000-5000))*100), end = '\r')
    tokens = tokenizer(df_title['Title'][num-1])

    out_list=predict_tag(tokens)
    temp=None
    jump = False
    for i in range(min(len(out_list),len(tokens))):
        if jump:
            jump = False 
            continue
        if tokens[i].lower() == 'hand' and i != len(tokens)-1 and   tokens[i+1].lower() == 'bag':
            jump = True
            submission['Record number'].append(num)
            submission['Aspect Name'].append('Type')
            submission['Aspect Value'].append(tokens[i] + ' ' + tokens[i+1])
            continue
        
        
        if tokens[i].lower() == 'cc' and i != len(tokens)-1 and   tokens[i+1].lower() == 'logo':
            jump = True
            submission['Record number'].append(num)
            submission['Aspect Name'].append('Accents')
            submission['Aspect Value'].append(tokens[i] + ' ' + tokens[i+1])
            continue
        if tokens[i].lower() == 'leather' and i != 0 and   out_list[i-1] == 'Material':
            submission['Aspect Value'][-1] = tokens[i-1] + ' ' + tokens[i]
            continue
        if out_list[i] == temp and (out_list[i] == 'Model' or
                                    out_list[i] == 'Brand' or 
                                    out_list[i] == 'Theme' or
                                    out_list[i] == 'Trim Material' or
                                    out_list[i] == 'Strap Drop' or 
                                    out_list[i] == 'Character' or
                                    out_list[i] == 'Character Family' or
                                    out_list[i] == 'Features' or
                                    out_list[i] == 'Handle Drop' or
                                    out_list[i] == 'Shoulder Strap' or
                                    out_list[i] == 'Hardware Material' or
                                    out_list[i] =='Handle Style' or
                                    out_list[i] =='Measurement, dimension' or
                                    out_list[i] == 'Pocket Type' or
                                    out_list[i] == 'Product Line'):
            prev_name = submission['Aspect Value'][-1]
            submission['Record number'].append(num)
            submission['Aspect Name'].append(out_list[i])
            submission['Aspect Value'].append(prev_name + ' ' + tokens[i])
        else:
            submission['Record number'].append(num)
            submission['Aspect Name'].append(out_list[i])
            submission['Aspect Value'].append(tokens[i])
            temp = out_list[i]
submission= pd.DataFrame(submission)
submission.to_csv('submission.csv', sep ='\t', index = False,header=None)

In [154]:
num = 280024
print(df_title['Title'][num])
tokens = tokenizer(df_title['Title'][num])

print(tokens)
out_list=predict_tag(tokens)
print(out_list)
# print(y_test)

KEEN Montclair Mini Brown Khaki Crosshatched Pattern Travel Crossbody Bag Purse
['KEEN', 'Montclair', 'Mini', 'Brown', 'Khaki', 'Crosshatched', 'Pattern', 'Travel', 'Crossbody', 'Bag', 'Purse']
['Size', 'Color', 'Color', 'Pattern', 'No Tag', 'Occasion', 'Type']
